In [2]:
import pandas as pd
import struct
import datetime
import pytz 
from datetime import datetime
import os

In [3]:
def to_str(x, lens):
    x_str = "{:.2f}".format(x)
    if (x // 100) == 0:
        # lon=format(lon, '.2f')
        return x_str.zfill(lens)

    elif ((-x) // 100) == 0:
        # lon = format(lon, '.2f')
        return x_str.zfill(lens + 1)
    else:
        return x_str

In [4]:
df_station=pd.read_csv("station.csv")
df_station['Region']=df_station['Region'].fillna('')
df_station['Country2']=df_station['Country2'].fillna('')
df_station['ElevationMeters']=df_station['ElevationMeters'].astype(int)
df_station['Region']=df_station['Region'].astype(str)
df_station['Country2']=df_station['Country2'].astype(str)
df_station.reset_index(inplace=True)
df_station['WhoAmI'] = df_station['Longitude'].apply(to_str, args=(6,)) + '_' + df_station['Latitude'].apply(to_str, args=(5,))
df_station['WhoAmI'].drop_duplicates(inplace=True)
df_station = df_station.sort_values(by=['WhoAmI'])
LOC=df_station['WhoAmI'].nunique()
df_station

,index,WhoAmI,Longitude,Latitude,ElevationMeters,ICAO,WMO,Region,Country2
38496,38496,-060.00_24.00,-60.0,24.00,0,NaN,0,,
38495,38495,-060.00_24.25,-60.0,24.25,0,NaN,0,,
38494,38494,-060.00_24.50,-60.0,24.50,0,NaN,0,,
38493,38493,-060.00_24.75,-60.0,24.75,0,NaN,0,,
38492,38492,-060.00_25.00,-60.0,25.00,0,NaN,0,,
...,...,...,...,...,...,...,...,...,...
4,4,-130.00_57.00,-130.0,57.00,1277,NaN,0,BC,CA
3,3,-130.00_57.25,-130.0,57.25,1429,NaN,0,BC,CA
2,2,-130.00_57.50,-130.0,57.50,1454,NaN,0,BC,CA
1,1,-130.00_57.75,-130.0,57.75,1345,NaN,0,BC,CA


In [79]:
# Define the conversion function # Excel's epoch starts on "January 0, 1900"
def datetime_to_excel_double(date_obj):
    excel_epoch = datetime(1899, 12, 31, tzinfo=pytz.UTC)  # Make it timezone-aware by setting the timezone to UTC
    delta = date_obj - excel_epoch
    excel_date = delta.days + (delta.seconds / 86400.0) + (date_obj.microsecond / 86400e6)+1
    return excel_date


file_path = "197605.csv" ###############################################This is the only place needs to be changed manually for all
df=pd.read_csv(file_path) 
# Fill NaN values with the previous non-null value
df['UTCISO8601'] = df['UTCISO8601'].fillna(method='ffill')
df = df.fillna(method='ffill') #just for 197605
# Convert UTCISO8601 to timestamp
df['UTCISO8601'] = pd.to_datetime(df['UTCISO8601'])
df['excel_double_datetime'] = df['UTCISO8601'].apply(datetime_to_excel_double)
df = df.sort_values(by=['excel_double_datetime', 'WhoAmI'])
COUNT=df['excel_double_datetime'].nunique()
unique_dates = df['UTCISO8601'].unique()
df['tempF102'] = df['tempF'] + 115
df['DewPointF104'] = df['DewPointF'] + 115
df['WindDirection107'] = df['WindDirection'] /5
df['GlobalHorizontalIrradianceWM2_120'] = df['GlobalHorizontalIrradianceWM2'] /5
df['DirectHorizontalIrradianceWM2_121'] = df['DirectHorizontalIrradianceWM2'] /5
df['WindDirection107'] = df['WindDirection107'].astype(int)
df['GlobalHorizontalIrradianceWM2_120'] = df['GlobalHorizontalIrradianceWM2_120'].astype(int)
df['DirectHorizontalIrradianceWM2_121'] = df['DirectHorizontalIrradianceWM2_121'].astype(int)
df['tempF102']=df['tempF102'].astype(int) #just for 197605
df['WindSpeedmph']=df['WindSpeedmph'].astype(int) #just for 197605
df['CloudCoverPerc']=df['CloudCoverPerc'].astype(int) #just for 197605
df['WindSpeed100mph']=df['WindSpeed100mph'].astype(int) #just for 197605
df

,Unnamed: 0,UTCISO8601,WhoAmI,DewPointF,tempF,GlobalHorizontalIrradianceWM2,CloudCoverPerc,DirectHorizontalIrradianceWM2,WindSpeedmph,WindDirection,WindSpeed100mph,excel_double_datetime,tempF102,DewPointF104,WindDirection107,GlobalHorizontalIrradianceWM2_120,DirectHorizontalIrradianceWM2_121
38484,28641024,1976-05-01 00:00:00+00:00,-060.00_24.00,67,73.0,0.0,70,0.0,4,198.0,5,27881.000000,188,182,39,0,0
38403,28640280,1976-05-01 00:00:00+00:00,-060.00_24.25,66,73.0,0.0,55,0.0,4,202.0,4,27881.000000,188,181,40,0,0
38462,28639536,1976-05-01 00:00:00+00:00,-060.00_24.50,66,73.0,0.0,79,0.0,2,218.0,2,27881.000000,188,181,43,0,0
38438,28638792,1976-05-01 00:00:00+00:00,-060.00_24.75,66,72.0,0.0,83,0.0,1,306.0,0,27881.000000,187,181,61,0,0
38422,28638048,1976-05-01 00:00:00+00:00,-060.00_25.00,66,72.0,0.0,60,0.0,2,14.0,2,27881.000000,187,181,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17594031,15337,1976-05-20 01:00:00+00:00,-130.00_53.00,36,45.0,559.0,56,470.0,11,236.0,13,27900.041667,160,151,47,111,94
17594643,2689,1976-05-20 01:00:00+00:00,-130.00_57.25,23,36.0,550.0,65,408.0,3,146.0,5,27900.041667,151,138,29,110,81
17599061,1945,1976-05-20 01:00:00+00:00,-130.00_57.50,23,36.0,559.0,83,418.0,4,153.0,6,27900.041667,151,138,30,111,83
17596688,1201,1976-05-20 01:00:00+00:00,-130.00_57.75,25,39.0,555.0,97,419.0,3,156.0,5,27900.041667,154,140,31,111,83


In [73]:
# Extract filename without extension
file_name = os.path.basename(file_path)  
file_name_without_extension = os.path.splitext(file_name)[0]  
file_name_without_extension

'197605'

In [7]:
def to_ascii_null_terminated(s):
    try:
        # Encode to ASCII and add a null terminator
        return s.encode('ascii') + b'\x00'
    except UnicodeEncodeError:
        # Handle the case where the string cannot be encoded in ASCII
        # This will replace non-ASCII characters with '?', and then add a null terminator
        return s.encode('ascii', 'replace') + b'\x00'

    
    
df_station['ascii_null_terminated_WhoAmI'] = df_station['WhoAmI'].apply(to_ascii_null_terminated)   
df_station['ascii_null_terminated_Region'] = df_station['Region'].apply(to_ascii_null_terminated) 
df_station['ascii_null_terminated_Country2'] = df_station['Country2'].apply(to_ascii_null_terminated) 
df_station


,index,WhoAmI,Longitude,Latitude,ElevationMeters,ICAO,WMO,Region,Country2,ascii_null_terminated_WhoAmI,ascii_null_terminated_Region,ascii_null_terminated_Country2
38496,38496,-060.00_24.00,-60.0,24.00,0,NaN,0,,,b'-060.00_24.00\x00',b'\x00',b'\x00'
38495,38495,-060.00_24.25,-60.0,24.25,0,NaN,0,,,b'-060.00_24.25\x00',b'\x00',b'\x00'
38494,38494,-060.00_24.50,-60.0,24.50,0,NaN,0,,,b'-060.00_24.50\x00',b'\x00',b'\x00'
38493,38493,-060.00_24.75,-60.0,24.75,0,NaN,0,,,b'-060.00_24.75\x00',b'\x00',b'\x00'
38492,38492,-060.00_25.00,-60.0,25.00,0,NaN,0,,,b'-060.00_25.00\x00',b'\x00',b'\x00'
...,...,...,...,...,...,...,...,...,...,...,...,...
4,4,-130.00_57.00,-130.0,57.00,1277,NaN,0,BC,CA,b'-130.00_57.00\x00',b'BC\x00',b'CA\x00'
3,3,-130.00_57.25,-130.0,57.25,1429,NaN,0,BC,CA,b'-130.00_57.25\x00',b'BC\x00',b'CA\x00'
2,2,-130.00_57.50,-130.0,57.50,1454,NaN,0,BC,CA,b'-130.00_57.50\x00',b'BC\x00',b'CA\x00'
1,1,-130.00_57.75,-130.0,57.75,1345,NaN,0,BC,CA,b'-130.00_57.75\x00',b'BC\x00',b'CA\x00'


In [80]:
aPWWVersion = 1
aPWWFileName = file_name_without_extension + ".pww"  

# Extracting the smallest start time and the largest end time from the DataFrame and converting them to timestamps
aStartDateTimeUTC = df['excel_double_datetime'].min()
aEndDateTimeUTC = df['excel_double_datetime'].max()
#area=[58, -130, 24, -60] North 58°, West -130°, South 24°, East -60°  
aMinLat=-130
aMaxLat=-60
aMinLon=24
aMaxLon=58
# Define the optional identifier field count
LOC_FC = 0  # for extra loc variables from table 1
VARCOUNT = 8  # Set this to the number of weather variable types you have

    
    
with open(aPWWFileName, 'wb') as file:
    file.write(struct.pack('<h', 2001))
    file.write(struct.pack('<h', 8065))
    file.write(struct.pack('<h', aPWWVersion))
    file.write(struct.pack('<d', aStartDateTimeUTC))
    file.write(struct.pack('<d', aEndDateTimeUTC))
    file.write(struct.pack('<d', aMinLat))
    file.write(struct.pack('<d', aMaxLat))
    file.write(struct.pack('<d', aMinLon))
    file.write(struct.pack('<d', aMaxLon))
    file.write(struct.pack('<h', 0))
    file.write(struct.pack('<i', COUNT))    #countNumber of datetime values (COUNT)
    file.write(struct.pack('<i', 3600))
    file.write(struct.pack('<i', LOC)) #Number of weather measurement locations (LOC)
    file.write(struct.pack('<h', LOC_FC)) #Loc_FC # Pack the data into INT16 format and write to stream
    file.write(struct.pack('<h', VARCOUNT))


    
    # Temp in F
    file.write(struct.pack('<h', 102))
    # Dew point in F
    file.write(struct.pack('<h', 104))
    # Wind speed at surface (10m) in mph
    file.write(struct.pack('<h', 106))
    # Wind direction at surface (10m) in 5-degree increments
    file.write(struct.pack('<h', 107))
    # Total cloud cover percentage
    file.write(struct.pack('<h', 119))
    # Wind speed at 100m in mph
    file.write(struct.pack('<h', 110))
    # Global Horizontal Irradiance in W/m^2 divided by 4
    file.write(struct.pack('<h', 120))
    # Direct Horizontal Irradiance in W/m^2 divided by 4
    file.write(struct.pack('<h', 121))
    file.write(struct.pack('<h', 8)) #BYTECOUNT
    for row in df_station.index:
        # Write Latitude (DOUBLE)
        file.write(struct.pack('<d', df_station['Latitude'][row]))
        # Write Longitude (DOUBLE)
        file.write(struct.pack('<d', df_station['Longitude'][row])) 
        # Write AltitudeM (INT16)
        file.write(struct.pack('<h', df_station['ElevationMeters'][row]))
        # Write Name (STRING)
        file.write(df_station['ascii_null_terminated_WhoAmI'][row])
        file.write(df_station['ascii_null_terminated_Country2'][row])
        file.write(df_station['ascii_null_terminated_Region'][row])

    for date in unique_dates:
        # Filter rows by unique date
        rows = df[df['UTCISO8601'] == date]
        
        for temp in rows['tempF102']:
            #file.write(temp.encode('utf-8'))
            #file.write(struct.pack('<i', temp))
            file.write(temp.to_bytes(1, 'little'))
        
        for dew_point in rows['DewPointF104']:
            #file.write(struct.pack('<b', dew_point.to_bytes(1, 'little')))
            file.write(dew_point.to_bytes(1, 'little'))
        for wind_speed in rows['WindSpeedmph']:
            #file.write(struct.pack('<b', wind_speed.to_bytes(1, 'little')))
            file.write(wind_speed.to_bytes(1, 'little'))
       
        for wind_direction in rows['WindDirection107']:
            #file.write(struct.pack('d', wind_direction))
            file.write(wind_direction.to_bytes(1, 'little'))
                  
        for CloudCoverPerc in rows['CloudCoverPerc']:
            #file.write(struct.pack('<b', CloudCoverPerc.to_bytes(1, 'little')))
            file.write(CloudCoverPerc.to_bytes(1, 'little'))
        
        for WindSpeed100mph in rows['WindSpeed100mph']:
            #file.write(struct.pack('<b', WindSpeed100mph.to_bytes(1, 'little')))
            file.write(WindSpeed100mph.to_bytes(1, 'little'))
           
        for GlobalHorizontalIrradianceWM2_120 in rows['GlobalHorizontalIrradianceWM2_120']:
            #file.write(struct.pack('d', GlobalHorizontalIrradianceWM2_120))
            file.write(GlobalHorizontalIrradianceWM2_120.to_bytes(1, 'little'))
        
        for DirectHorizontalIrradianceWM2_121 in rows['DirectHorizontalIrradianceWM2_121']:
            #file.write(struct.pack('d', DirectHorizontalIrradianceWM2_121))
            file.write(DirectHorizontalIrradianceWM2_121.to_bytes(1, 'little'))

   
    